<a href="https://colab.research.google.com/github/Tulsi97/ERAV1_dev/blob/develop/ERA_V1_S2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the necessary modules from the 
# PyTorch library and torchvision for working 
# with neural networks, datasets, and transformations.

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
!pip install torchsummary
from torchsummary import summary

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
# Checking if CUDA is available for GPU acceleration.
use_cuda = torch.cuda.is_available()
# Setting the device to either CUDA (GPU) if available or CPU if not.
device = torch.device("cuda" if use_cuda else "cpu")
device

device(type='cuda')

In [ ]:
# Setting the batch size for data loading and training.
batch_size = 128
# Creating a data loader for the training dataset MNIST. 
# Downloading the dataset if not available, applying transformations
# (converts to tensor and normalizes), and shuffling the data.
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)
# Creating a data loader for the testing dataset MNIST. 
# Using the same transformations as the training dataset but setting the train flag to False.
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True)

# Some Notes on our naive model

We are going to write a network based on what we have learnt so far. 

The size of the input image is 28x28x1. We are going to add as many layers as required to reach RF = 32 "atleast". 

In [ ]:
class FirstDNN(nn.Module):
    def __init__(self):
        super(FirstDNN, self).__init__()
        # Define the layers of the neural network
        
        # Convolutional layer 1: input channels=1, output channels=32, kernel size=3, padding=1
        # r_in:1, n_in:28, j_in:1, s:1, r_out:3, n_out:28, j_out:1
        self.conv1 = nn.Conv2d(1, 32, 3, padding=1)
        
        # Convolutional layer 2: input channels=32, output channels=64, kernel size=3, padding=1
        # r_in:3 , n_in:28 , j_in:1 , s:1 , r_out:5 , n_out:28 , j_out:1
        self.conv2 = nn.Conv2d(32, 64, 3, padding=1)
        
        # Max pooling layer 1: kernel size=2, stride=2
        # r_in:5 , n_in:28 , j_in:1 , s:2 , r_out:6 , n_out:14 , j_out:2
        self.pool1 = nn.MaxPool2d(2, 2)
        
        # Convolutional layer 3: input channels=64, output channels=128, kernel size=3, padding=1
        # r_in:6 , n_in:14 , j_in:2 , s:1 , r_out:10 , n_out:14 , j_out:2
        self.conv3 = nn.Conv2d(64, 128, 3, padding=1)
        
        # Convolutional layer 4: input channels=128, output channels=256, kernel size=3, padding=1
        # r_in:10 , n_in:14 , j_in:2 , s:1 , r_out:14 , n_out:14 , j_out:2
        self.conv4 = nn.Conv2d(128, 256, 3, padding=1)
        
        # Max pooling layer 2: kernel size=2, stride=2
        # r_in:14 , n_in:14 , j_in:2 , s:2 , r_out:16 , n_out:7 , j_out:4
        self.pool2 = nn.MaxPool2d(2, 2)
        
        # Convolutional layer 5: input channels=256, output channels=512, kernel size=3
        # r_in:16 , n_in:7 , j_in:4 , s:1 , r_out:24 , n_out:5 , j_out:4
        self.conv5 = nn.Conv2d(256, 512, 3)
        
        # Convolutional layer 6: input channels=512, output channels=1024, kernel size=3
        # r_in:24 , n_in:5 , j_in:4 , s:1 , r_out:32 , n_out:3 , j_out:4
        self.conv6 = nn.Conv2d(512, 1024, 3)
        
        # Convolutional layer 7: input channels=1024, output channels=10, kernel size=3
        # r_in:32 , n_in:3 , j_in:4 , s:1 , r_out:40 , n_out:1 , j_out:4
        self.conv7 = nn.Conv2d(1024, 10, 3)

    def forward(self, x):
        # Define the forward pass of the neural network
        
        # Apply convolutional layer 1, followed by ReLU activation function, and then max pooling
        x = self.pool1(F.relu(self.conv2(F.relu(self.conv1(x)))))
        
        # Apply convolutional layer 3, followed by ReLU activation function, and then max pooling
        x = self.pool2(F.relu(self.conv4(F.relu(self.conv3(x)))))
        
        # Apply convolutional layer 5, followed by ReLU activation function
        x = F.relu(self.conv6(F.relu(self.conv5(x))))
        
        # Apply convolutional layer 7, followed by ReLU activation function
        x = F.relu(self.conv7(x))
        
        # Reshape the tensor to have a shape of (-1, 10)
        x = x.view(-1, 10)
        
        # Apply the log softmax activation function
        return F.log_softmax(x)

In [ ]:
# Creating object of class FirstDNN and adding it to the device specified above
model = FirstDNN().to(device)

In [ ]:
# looking at the layer wise output dimensions and number of parameters for a given input size
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 28, 28]             320
            Conv2d-2           [-1, 64, 28, 28]          18,496
         MaxPool2d-3           [-1, 64, 14, 14]               0
            Conv2d-4          [-1, 128, 14, 14]          73,856
            Conv2d-5          [-1, 256, 14, 14]         295,168
         MaxPool2d-6            [-1, 256, 7, 7]               0
            Conv2d-7            [-1, 512, 5, 5]       1,180,160
            Conv2d-8           [-1, 1024, 3, 3]       4,719,616
            Conv2d-9             [-1, 10, 1, 1]          92,170
Total params: 6,379,786
Trainable params: 6,379,786
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Forward/backward pass size (MB): 1.51
Params size (MB): 24.34
Estimated Total Size (MB): 25.85
-------------------------------------

<ipython-input-4-07eb6d900f57>:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


In [ ]:
from tqdm import tqdm

def train(model, device, train_loader, optimizer, epoch):
    # Train the model
    model.train()
    
    # Create a progress bar for visualization
    pbar = tqdm(train_loader)
    
    # Iterate over the batches of data
    for batch_idx, (data, target) in enumerate(pbar):
        # Move the data and target tensors to the device (GPU or CPU)
        data, target = data.to(device), target.to(device)
        
        # Reset the gradients of the optimizer
        optimizer.zero_grad()
        
        # Perform forward pass to get the model's output
        output = model(data)
        
        # Calculate the loss using negative log likelihood loss
        loss = F.nll_loss(output, target)
        
        # Perform backward pass to compute the gradients of the parameters
        loss.backward()
        
        # Update the parameters of the model using the gradients
        optimizer.step()
        
        # Update the progress bar description with current loss and batch index
        pbar.set_description(desc=f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    # Evaluate the model on the test set
    model.eval()
    
    # Initialize variables for tracking test loss and accuracy
    test_loss = 0
    correct = 0
    
    # Disable gradient calculation as we are only evaluating the model
    with torch.no_grad():
        # Iterate over the batches of data in the test set
        for data, target in test_loader:
            # Move the data and target tensors to the device (GPU or CPU)
            data, target = data.to(device), target.to(device)
            
            # Perform forward pass to get the model's output
            output = model(data)
            
            # Calculate the loss using negative log likelihood loss, summing up the batch losses
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            
            # Get the predicted labels by finding the index of the maximum log-probability
            pred = output.argmax(dim=1, keepdim=True)
            
            # Count the number of correct predictions by comparing predicted labels with target labels
            correct += pred.eq(target.view_as(pred)).sum().item()
    
    # Calculate the average test loss
    test_loss /= len(test_loader.dataset)
    
    # Print the test set results: average loss and accuracy
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), 100. * correct / len(test_loader.dataset)))


In [ ]:
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
# Create an optimizer (Stochastic Gradient Descent) and pass the model parameters,
# learning rate (lr) of 0.01, and momentum of 0.9

for epoch in range(1, 2):
    # Iterate over the specified range of epochs (in this case, 1 to 1)
    
    train(model, device, train_loader, optimizer, epoch)
    # Call the train function to train the model on the training data.
    # Pass the model, device, train_loader, optimizer, and epoch as arguments
    
    test(model, device, test_loader)
    # Call the test function to evaluate the model on the test data.
    # Pass the model, device, and test_loader as arguments

  0%|          | 0/469 [00:00<?, ?it/s]<ipython-input-4-07eb6d900f57>:61: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)
loss=0.06011991202831268 batch_id=468: 100%|██████████| 469/469 [00:34<00:00, 13.65it/s]



Test set: Average loss: 0.0556, Accuracy: 9821/10000 (98%)

